In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.io import loadmat
from os.path import join as opj
import os
from scipy.stats import pearsonr

spm_file1 = r'/mnt/workdir/DCM/BIDS/derivatives/Nipype/working_dir/game1/separate_hexagon/Setall' \
            r'/6fold/work_1st/_subj_id_099/level1estimate/SPM.mat'
spm_file2 = r'/mnt/workdir/DCM/BIDS/derivatives/SPM/sub-099/SPM.mat'

spmmat1 = loadmat(spm_file1,struct_as_record=False)
spmmat2 = loadmat(spm_file2,struct_as_record=False)

In [4]:
designMatrix1 = spmmat1['SPM'][0][0].xX[0][0].X
designMatrix2 = spmmat2['SPM'][0][0].xX[0][0].X

designMatrix1 = np.round(designMatrix1.astype('float32'),decimals=5)
designMatrix2 = np.round(designMatrix2.astype('float32'),decimals=5)

names1 = [i[0] for i in spmmat1['SPM'][0][0].xX[0][0].name[0]]
names2 = [i[0] for i in spmmat2['SPM'][0][0].xX[0][0].name[0]]

print((designMatrix1==designMatrix2).all())

from scipy.stats import pearsonr
r,p = pearsonr(designMatrix1.reshape(-1),designMatrix2.reshape(-1))
print(r,p)

False
0.9999999999982052 0.0


In [5]:
designMatrix1 = spmmat1['SPM'][0][0].xX[0][0].nKX
designMatrix2 = spmmat2['SPM'][0][0].xX[0][0].nKX

designMatrix1 = np.round(designMatrix1.astype('float32'),decimals=5)
designMatrix2 = np.round(designMatrix2.astype('float32'),decimals=5)

names1 = [i[0] for i in spmmat1['SPM'][0][0].xX[0][0].name[0]]
names2 = [i[0] for i in spmmat2['SPM'][0][0].xX[0][0].name[0]]

print((designMatrix1==designMatrix2).all())
design_matrix1 = pd.DataFrame(designMatrix1, columns=names1)
design_matrix2 = pd.DataFrame(designMatrix2, columns=names2)

r,p = pearsonr(designMatrix1.reshape(-1),designMatrix2.reshape(-1))
print(r,p)

False
0.9999537970709492 0.0


In [7]:
diff_matrix = designMatrix1 == designMatrix2
for i,c in zip(range(102),names1):
    sum_num = diff_matrix[:,i].sum()
    if sum_num != 980:
        print(c)
        print(sum_num)

Sn(1) M1*bf(1)
816
Sn(1) M2_corr*bf(1)
816
Sn(1) M2_corrxcos^1*bf(1)
816
Sn(1) M2_corrxsin^1*bf(1)
818
Sn(1) M2_error*bf(1)
817
Sn(1) decision_corr*bf(1)
816
Sn(1) decision_corrxcos^1*bf(1)
816
Sn(1) decision_corrxsin^1*bf(1)
817
Sn(1) decision_error*bf(1)
822
Sn(1) pressButton*bf(1)
816
Sn(1) trans_x
817
Sn(1) trans_y
816
Sn(1) trans_z
816
Sn(1) rot_x
817
Sn(1) rot_y
816
Sn(1) rot_z
816
Sn(2) M1*bf(1)
817
Sn(2) M2_corr*bf(1)
817
Sn(2) M2_corrxcos^1*bf(1)
818
Sn(2) M2_corrxsin^1*bf(1)
817
Sn(2) M2_error*bf(1)
817
Sn(2) decision_corr*bf(1)
817
Sn(2) decision_corrxcos^1*bf(1)
818
Sn(2) decision_corrxsin^1*bf(1)
817
Sn(2) decision_error*bf(1)
817
Sn(2) pressButton*bf(1)
817
Sn(2) trans_x
817
Sn(2) trans_y
819
Sn(2) trans_z
817
Sn(2) rot_x
817
Sn(2) rot_y
819
Sn(2) rot_z
819
Sn(3) M1*bf(1)
817
Sn(3) M2_corr*bf(1)
817
Sn(3) M2_corrxcos^1*bf(1)
817
Sn(3) M2_corrxsin^1*bf(1)
817
Sn(3) M2_error*bf(1)
818
Sn(3) decision_corr*bf(1)
819
Sn(3) decision_corrxcos^1*bf(1)
817
Sn(3) decision_corrxsin^

## The similarity of the beta map

In [8]:
import os
from nilearn.image import load_img
from nilearn.masking import apply_mask

nipype_dir = r'/mnt/workdir/DCM/BIDS/derivatives/Nipype/working_dir/game1/separate_hexagon/Setall/' \
             r'6fold/work_1st/_subj_id_099/level1estimate'
spm_dir = r'/mnt/workdir/DCM/BIDS/derivatives/SPM/sub-099'
mask = r'/mnt/data/Template/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii'
similarity = {}
for i in range(1,103):
    i = str(i).zfill(4)

    nipype_beta_map = apply_mask(os.path.join(nipype_dir,'beta_{}.nii'.format(i)),mask)
    mean1 = nipype_beta_map.mean()
    std1 = nipype_beta_map.std()
    spm_beta_map =  apply_mask(os.path.join(spm_dir,'beta_{}.nii'.format(i)),mask)
    mean2 = spm_beta_map.mean()
    std2 = spm_beta_map.std()

    r,p = pearsonr(nipype_beta_map,spm_beta_map)
    similarity[f'beta_{i}'] = r
    print(f'beta_{i}:','mean',mean1,mean2,'std',std1,std2)

beta_0001: mean 0.015124963 0.011804107 std 0.64233327 0.6497377
beta_0002: mean 0.042541537 0.03289993 std 0.72810304 0.73637784
beta_0003: mean -0.00011478467 -0.0016897997 std 0.23470081 0.24075687
beta_0004: mean -0.23557155 -0.2412842 std 0.33011204 0.33423725
beta_0005: mean -0.19088416 -0.20572664 std 1.2248027 1.236347
beta_0006: mean 0.117603704 0.09915303 std 1.9425739 1.9557177
beta_0007: mean 0.15168492 0.17022325 std 0.6390823 0.6571726
beta_0008: mean -0.10707307 -0.11942491 std 0.73713 0.7579712
beta_0009: mean 0.5603787 0.5648691 std 1.6428578 1.6506649
beta_0010: mean 0.36258823 0.3694813 std 2.1846957 2.2210846
beta_0011: mean 3.5583766 3.6446936 std 5.4541187 5.545212
beta_0012: mean 1.8318471 1.9000096 std 6.2740126 6.406802
beta_0013: mean -1.5644807 -1.5531794 std 5.598741 5.8039355
beta_0014: mean 206.70341 213.42117 std 327.23373 332.08765
beta_0015: mean -134.9707 -139.11433 std 364.0146 371.90933
beta_0016: mean -6.4100647 -8.977177 std 320.41803 328.66232
bet